In [32]:
import re
pattern = re.compile("\t+")

rdd = sc.parallelize([
    u"foo\t\t\t\tbar",
    u"123\t\t\t456\t\t789\t0"
])

rdd.map(lambda x: pattern.split(x)).collect()

[[u'foo', u'bar'], [u'123', u'456', u'789', u'0']]

In [111]:
book1 = sc.textFile("file:/home/training/Desktop/Harry_Potter/1.Harry Potter and the Sorcerer's Stone.txt")

In [112]:
book1 = book1.map(lambda line:line.split(" "))

In [347]:
book1.take(2)

[[u'CHAPTER', u'ONE'], [u'THE', u'BOY', u'WHO', u'LIVED']]

In [117]:
# join the text

for r in book1.take(5):
    print " ".join(r)

CHAPTER ONE
THE BOY WHO LIVED
Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much. They were the last people you'd expect to be involved in anything strange or mysterious, because they just didn't hold with such nonsense.
Mr. Dursley was the director of a firm called Grunnings, which made drills. He was a big, beefy man with hardly any neck, although he did have a very large mustache. Mrs. Dursley was thin and blonde and had nearly twice the usual amount of neck, which came in very useful as she spent so much of her time craning over garden fences, spying on the neighbors. The Dursleys had a small son called Dudley and in their opinion there was no finer boy anywhere.
The Dursleys had everything they wanted, but they also had a secret, and their greatest fear was that somebody would discover it. They didn't think they could bear it if anyone found out about the Potters. Mrs. Potter was Mrs. Dursley's sister, but the

In [115]:
#total book1 word count
book1.count()

3047

In [116]:
#total book1 distinct word count
book1.map(lambda r:r[0]).distinct().count()

849

In [64]:
#filter sentence starting with Harry
harry = sc.textFile("file:/home/training/Desktop/Harry_Potter/1.Harry Potter and the Sorcerer's Stone.txt").filter(lambda line: line.startswith('Harry'))

In [70]:
harry.take(5)

[u'Harry woke with a start. His aunt rapped on the door again.',
 u'Harry groaned.',
 u'Harry was frying eggs by the time Dudley arrived in the kitchen with his mother. Dudley looked a lot like Uncle Vernon. He had a large pink face, not much neck, small, watery blue eyes, and thick blond hair that lay smoothly on his thick, fat head. Aunt Petunia often said that Dudley looked like a baby angel -- Harry often said that Dudley looked like a pig in a wig.',
 u"Harry put the plates of egg and bacon on the table, which was difficult as there wasn't much room. Dudley, meanwhile, was counting his presents. His face fell.",
 u"Harry had the best morning he'd had in a long time. He was careful to walk a little way apart from the Dursleys so that Dudley and Piers, who were starting to get bored with the animals by lunchtime, wouldn't fall back on their favorite hobby of hitting him. They ate in the zoo restaurant, and when Dudley had a tantrum because his knickerbocker glory didn't have enough 

In [120]:
#print distinct words and flatten them
book1_flatmap = sc.textFile("file:/home/training/Desktop/Harry_Potter/1.Harry Potter and the Sorcerer's Stone.txt").flatMap(lambda line: line.split()).distinct()

In [121]:
book1_flatmap.take(20)

[u'"--and',
 u'ready!"',
 u'shouted,',
 u'wrought-iron',
 u'shouted.',
 u'yellow',
 u'four',
 u'Does',
 u'unwrapped.',
 u'spiders',
 u'hanging',
 u'clearing.',
 u'screaming.',
 u'Until',
 u'fiddling',
 u'wand."',
 u'"howling',
 u'wizardry',
 u'cake,',
 u'Ronald']

In [80]:
#count distinct words in book 1
sc.textFile("file:/home/training/Desktop/Harry_Potter/1.Harry Potter and the Sorcerer's Stone.txt").flatMap(lambda line: line.split()).distinct().count()

11897

In [127]:
#** apply lower() and remove punctuation **
import re
def removePunctuation(text):
    """Removes punctuation, changes to lowercase, and strips leading and trailing spaces.
    Note:
        Only spaces, letters, and numbers should be retained.  Other characters should should be
        eliminated. (e.g. it's becomes its)
    Args:
        text (str): A string.
    Returns:
        str: The cleaned up string.
    """
    return re.sub('[^a-z| |0-9]', '', text.strip().lower())
print removePunctuation("The Elephant's 4 cats. ")

the elephants 4 cats


In [197]:
#count frequency of words in book 1
book1words = sc.textFile("file:/home/training/Desktop/Harry_Potter/1.Harry Potter and the Sorcerer's Stone.txt").flatMap(lambda line: line.strip().split(" ")).filter(lambda x: x != '')
#apply punctuaion & lower function
book1words = book1words.map(removePunctuation)
#remove punctuation
print book1words.take(5)

#map to key-value pair
book1words_1 = book1words.map(lambda word : (word,1))
print book1words_1.take(5)


[u'chapter', u'one', u'the', u'boy', u'who']
[(u'chapter', 1), (u'one', 1), (u'the', 1), (u'boy', 1), (u'who', 1)]


In [150]:
#reduceByKey() operates by applying the function first within each partition on a per-key basis and then across the partitions, allowing it to scale efficiently to large datasets.
book1words_1_agg = book1words_1.reduceByKey(lambda a,b : a+b)
print book1words_1_agg.take(5)

[(u'', 852), (u'both', 32), (u'foul', 4), (u'four', 32), (u'woods', 7)]


In [ ]:
#remove stopwords
#book1words_1_agg_remove_stopwords = book1words_1_agg(lambda x : x not in )
#print book1words_1_agg.take(5)

In [151]:
#order by key (swap key & value first)
book1words_1_agg_order = book1words_1_agg.map(lambda (a,b): (b,a)).sortByKey(ascending=False)
print book1words_1_agg_order.take(100)

[(3627, u'the'), (1918, u'and'), (1856, u'to'), (1688, u'a'), (1527, u'he'), (1259, u'of'), (1213, u'harry'), (1186, u'was'), (1026, u'it'), (965, u'in'), (937, u'his'), (863, u'you'), (852, u''), (794, u'said'), (702, u'had'), (652, u'i'), (636, u'on'), (625, u'at'), (601, u'that'), (597, u'they'), (526, u'as'), (501, u'him'), (483, u'but'), (416, u'with'), (410, u'ron'), (397, u'all'), (372, u'out'), (369, u'for'), (368, u'up'), (367, u'be'), (342, u'what'), (341, u'were'), (336, u'hagrid'), (325, u'them'), (297, u'have'), (269, u'there'), (261, u'back'), (257, u'hermione'), (255, u'one'), (249, u'this'), (245, u'if'), (241, u'from'), (238, u'so'), (235, u'its'), (234, u'not'), (229, u'she'), (223, u'about'), (219, u'an'), (219, u'into'), (217, u'me'), (217, u'their'), (211, u'been'), (201, u'off'), (198, u'got'), (196, u'no'), (195, u'could'), (195, u'didnt'), (194, u'get'), (194, u'like'), (188, u'know'), (180, u'professor'), (180, u'see'), (180, u'just'), (180, u'down'), (179, u'h

In [222]:
# count all words & distinct words
print book1words.count()
print book1words.distinct().count()

78444
6037


In [122]:
#define wordcount function
def wordCount(wordListRDD):
    """Creates a pair RDD with word counts from an RDD of words.
    Args:
        wordListRDD (RDD of str): An RDD consisting of words.
    Returns:
        RDD of (str, int): An RDD consisting of (word, count) tuples.
    """
    return (wordListRDD.map(lambda x: (x,1)).reduceByKey(lambda x, y: x+y))

#print wordCount(wordsRDD).collect()

In [225]:
from pyspark import SparkConf, SparkContext
from pyspark import sql
sqlContext = sql.SQLContext(sc)

In [232]:
#! sudo pip install pandas
import pandas as pd
#import 
# $example on$
#from pyspark.ml.feature import NGram
# $example off$
#from pyspark.sql import SparkSession

In [238]:
#count specific words - turn to dataframe
book1df = pd.DataFrame(book1words_1_agg.collect())

In [242]:
#count - name columns
book1df.columns = ['word','count']
book1df.head()


,word,count
0,,852
1,both,32
2,foul,4
3,four,32
4,woods,7


In [313]:
#print book1df[['word','count']].reset_index().drop('index', 1).sort('count', ascending = False)

In [350]:
#book 1
print book1df[(book1df['word'] == ('harry')) | (book1df['word'] =='ron')|(book1df['word']=='hermione') \
              | (book1df['word']=='hagrid')|(book1df['word']=='malfoy') | (book1df['word']=='snape') \
              |(book1df['word']=='dumbledore') | (book1df['word']== 'voldemort') | (book1df['word']=='hogwarts')\
              | (book1df['word']=='azkaban') |(book1df['word']=='accio') | (book1df['word']== 'expelliarmus')\
              |(book1df['word']== 'stupefy') | (book1df['word']== 'lumos') |(book1df['word']== 'imperio')\
              | (book1df['word']== 'crucio') |(book1df['word']== 'avada') | (book1df['word']== 'kedavra')]


            word  count
440        snape    145
623    voldemort     31
765        harry   1213
1248    hogwarts     79
1348  dumbledore    143
2279    hermione    257
2525      malfoy    109
4897         ron    410
5728      hagrid    336


In [253]:
#use sqlcontext to create the same DataFrame
sfpdDF = sqlContext.createDataFrame(book1words_1_agg, ["word", "count"])

In [254]:
sfpdDF.printSchema()
sfpdDF.show(30)

root
 |-- word: string (nullable = true)
 |-- count: long (nullable = true)

word      count
          852  
both      32   
foul      4    
four      32   
woods     7    
spiders   2    
ornate    1    
wizardry  6    
lord      4    
flicking  1    
sinking   2    
figg      5    
yellow    5    
bringing  5    
disturb   2    
basics    1    
wooden    8    
wednesday 3    
specially 1    
tired     6    
hanging   12   
bacon     7    
second    43   
crisply   1    
sailed    2    
scraped   1    
thunder   1    
fingers   14   
pawed     2    
galleons  4    


In [349]:
#count harry
descDF = sfpdDF.filter(sfpdDF.word =='harry')
descDF.show()

word  count
harry 1213 
